**no care was taken to make sure this is theoretically/quantitatively correct** for a first pass. Produces uneven illumination (or uneven autofluorescence/background depending on how it's modified) and adds some shot-like noise on top of this to corrupt the signal. The goal is to simulate lower signal data, since our real starting sample data has high contrast and high SNR. 

In [379]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import math
import scipy.misc

sns.set_style('dark', rc={'image.cmap':'inferno'})

In [364]:
#parameters to be used later
illumination_unevenness = 0.8 #modifies the sigma determining the illumination artifact
illumination_unevenness_power = 4
mean_dark_current = 10**2 

In [346]:
from skimage.io import imread
data_nodrug = imread("../../../Downloads/DMSO-1.tif")

In [367]:
data_nodrug.dtype
data_nodrug.shape

im = data_nodrug[4,:,:,2]

plt.imshow(im, vmin = 0, vmax = im.max()*0.5)

In [368]:
mid_index = int(im.shape[0]/2)
print(mid_index)

In [369]:
#crudely simulate uneven illumination
mu = mid_index
sigma = mid_index*illumination_unevenness

def gaussian(x, mu, sigma):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))

gauss = gaussian(np.linspace(0, im.shape[0]-1, im.shape[1]), mu, sigma)

dim1 = im.shape[0]
dim2 = im.shape[1]
noiseD1 = np.zeros([dim1,dim2]) 

for x in range(0, dim1):
    noiseD1[:,x] = gauss

illumination = np.power(noiseD1.transpose()*noiseD1 , illumination_unevenness_power/2)

im_illum_artifact = illumination*im

plt.imshow(im_illum_artifact, vmin = 0, vmax = im_illum_artifact.max()*0.5)

In [370]:
#add dark/thermal noise, uniform across image and normally distributed
#not technically correct, noise should come from dark current shot noise
dark_current = np.random.normal(mean_dark_current, 1, [dim1,dim2])

In [371]:
noisy = im_illum_artifact + dark_current
noisy = noisy/noisy.max() * 2**16

In [372]:
plt.imshow(noisy, vmin = 0, vmax = noisy.max()*0.5)

In [380]:
mpimg.imsave("../data/uneven_illum_test.tif",noisy)